In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common import by
import time

def get_press_disclose(soup):
    press = ''
    disclose = ''
    
    for i in soup.find_all(class_='meeting-app-info-block mx-0'):

        if i.find(class_='meeting-app-info-header row').text == 'Session Schedule':
            press = i
        elif i.find(class_='meeting-app-info-header row').text == 'Disclosures':
            disclose = i

    return press, disclose

def clean_semicolon(st):
    return re.sub(';$|; $', '', st, flags=re.S)


driver = webdriver.Chrome()
driver.get('https://am2023.aad.org/sessions#')

In [ ]:
soupOut = BS(driver.page_source, 'html.parser')
all_links = []
for li in soupOut.find_all(class_='meeting-app-info-block')[3:]:
    
    url = li.find('a').get('href')
    session_url = f'https://am2023.aad.org{url}'
    all_links.append(session_url)
    
dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_id':[],
    'news_type':[],
    'session_title':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[],
    'handouts':[]
    }

In [ ]:
c = 0

for url in all_links[:109]:
    
    session_url = url
    driver.get(session_url)
    time.sleep(8)
    soup = BS(driver.page_source, 'html.parser')
    session_title = soup.find('h1').text

    # ------------------- date and time -------------------
    time_date = soup.find('span', class_='mx-2').parent
    session_date = time_date.text.strip().split('|')[0].strip()
    session_time = time_date.text.strip().split('|')[1].strip()

    # ------------------------- abstract text -------------------------
    session_text = ''
    for i in list(soup.find(class_='land-page-hr').next_siblings):
        if (re.search('Description', i.text) or re.search('Learning Objectives', i.text)) and list(i.children)[0].name == 'h2':
            for j in list(i.children):
                session_text += j.text.strip() + ' '



    # ---------------------- location, type, category -----------------------
    session_loca = ''
    session_type = ''
    session_cate = ''

    for i in soup.find(class_='col-lg-4 p-1').find(class_='meeting-app-info-body').find_all(class_='row meeting-app-info-item'):

        if re.search('Location', i.text):
            session_loca = i.text

        elif re.search('Type', i.text):
            for typ in i.find_all('span'):
                session_type += typ.text.strip() + '; '

        elif re.search('Categories', i.text):
            for cat in i.find_all('span'):
                session_cate += cat.text.strip() +'; '

    session_cate = clean_semicolon(session_cate)
    session_type = clean_semicolon(session_type)


    # --------------------------- authors -----------------
    session_author = ''
    handout_nhiMila = True
    for i in soup.find_all(class_='meeting-app-info-block'):

        if i.find(class_='meeting-app-info-header row').text == 'Directors/Co-Directors':

            for au in i.find_all(class_='row meeting-app-info-item'):
                session_author += au.text.strip() + '; '

        elif i.find(class_='meeting-app-info-header row').text == 'Speakers':
            for au in i.find_all(class_='row meeting-app-info-item'):
                session_author += au.text.strip() + '; '

        elif i.find(class_='meeting-app-info-header row').text == 'Handouts':
            dic['handouts'].append(i)
            handout_nhiMila = False

    if handout_nhiMila:
        dic['handouts'].append('')

    session_author = clean_semicolon(session_author)

    # ----------------------------- session disclosure ---------------------------
    sess_disclosure = ''

    if get_press_disclose(soup)[1]: # if disclosures are present
        for i in get_press_disclose(soup)[1].find_all(class_='row meeting-app-info-item'):
            auth = i.find('h5').text.strip()
            dis = i.find('div').text.strip()
            dis = dis.replace(';', ' ')

            sess_disclosure += auth + ' : ' + dis + '; '

    sess_disclosure = re.sub(';$|; $', '', sess_disclosure, flags=re.S)

    print(f'sess title :- {session_title}')
    print(f'sess date :- {session_date}')
    print(f'sess time :- {session_time}')
    print(f'sess auth :- {session_author}')
    print(f'sess loca :- {session_loca}')
    print(f'sess type :- {session_type}')
    print(f'sess cate :- {session_cate}')

    # print(f'sess text :- {session_text}')
    # print(f'sess diss :- {sess_disclosure}')

    dic['session_id'].append(f'S{c+1}')
    dic['news_type'].append('Session')
    dic['source_id'].append('')
    dic['manual_id'].append('')
    dic['article_title'].append(session_title)
    dic['url'].append(session_url)
    dic['authors'].append(session_author)
    dic['author_affiliation'].append('')
    dic['abstract_text'].append(session_text)
    dic['date'].append(session_date)
    dic['start_time'].append(session_time.split('-')[0].strip())
    dic['end_time'].append(session_time.split('-')[-1].strip())
    dic['location'].append(session_loca)
    dic['session_title'].append(session_title)
    dic['session_type'].append(session_type)
    dic['category'].append(session_cate)
    dic['sub_category'].append('')
    dic['disclosure'].append(sess_disclosure)

    print('========'*15)

    # --------------------- pressentaion --------------------------
    if get_press_disclose(soup)[0]: # if presentaions are present
        for i in get_press_disclose(soup)[0].find_all(class_='row meeting-app-info-item'):

            article_title = i.find(class_='m-0 font-weight-bold').text.strip()
            article_time = i.find(class_='col-md-3').text.strip()
            article_author = i.find(class_='font-italic m-0').text.strip()

            print(f'art title :- {article_title}')
            print(f'art time :- {article_time}')
            print(f'art author :- {article_author}')

            dic['session_id'].append(f'S{c+1}')
            dic['news_type'].append('Abstract')
            dic['source_id'].append('')
            dic['manual_id'].append('')
            dic['article_title'].append(article_title)
            dic['url'].append(session_url)
            dic['authors'].append(article_author)
            dic['author_affiliation'].append('')
            dic['abstract_text'].append('')
            dic['date'].append(session_date)
            dic['start_time'].append(article_time)
            dic['end_time'].append('')
            dic['location'].append(session_loca)
            dic['session_title'].append(session_title)
            dic['session_type'].append(session_type)
            dic['category'].append(session_cate)
            dic['sub_category'].append('')
            dic['disclosure'].append('')
            dic['handouts'].append('')
            print('*==*='*15)

    c += 1


In [ ]:
df = pd.DataFrame(dic)
df